# Gaussian Process Variational Autoencoder (GP-VAE)

A **Gaussian Process Variational Autoencoder (GP-VAE)** is a hybrid model that combines the power of **Variational Autoencoders (VAE)** with **Gaussian Processes (GP)** in the latent space to capture complex dependencies in data. This model is especially useful for applications that require modeling uncertainty and correlations, such as time-series forecasting, anomaly detection, and generative modeling.

### Key Components:

1. **Encoder**:
   - The encoder maps the input data $\\mathbf{x}$ to a probabilistic distribution over latent variables $\\mathbf{z}$. It typically outputs a **mean** and **variance** which define the distribution $q(\\mathbf{z} | \\mathbf{x})$ from which latent variables are sampled.
   - The encoder network is typically a neural network (fully connected layers) that produces these parameters based on the input data.

2. **Gaussian Process in Latent Space**:
   - Instead of assuming independent latent variables (as in traditional VAEs), GP-VAE incorporates a **Gaussian Process** prior in the latent space. This prior defines smooth correlations between latent variables. The GP prior captures the dependencies between latent variables based on a kernel function (e.g., the **RBF kernel**, also called **Exponentiated Quadratic**).
   - The GP prior enables the latent variables to exhibit smooth transitions and dependencies over time (temporal) or space (spatial), making the model suitable for time-series data or geospatial data.

3. **Decoder**:
   - The decoder takes the latent variables $\\mathbf{z}$ and generates the output data $\\mathbf{x}$. The decoder learns how to map the latent space back to the original data space using a neural network, just like in standard VAEs.

### Loss Function:

The **objective** during training is to maximize the **Evidence Lower Bound (ELBO)**. The ELBO consists of two main terms:

1. **Reconstruction Loss**:
   - This term measures how well the decoder reconstructs the input data from the latent variables $\\mathbf{z}$.
   - Typically, the reconstruction loss is computed as the **log-likelihood** of the data under the decoder's distribution $p(\\mathbf{x} | \\mathbf{z})$. For continuous data, this could be mean squared error (MSE), and for binary data, it might be binary cross-entropy.

$$ \\mathcal{L}_{\\text{recon}} = \\mathbb{E}_{q(\\mathbf{z} | \\mathbf{x})} \\left[ \\log p(\\mathbf{x} | \\mathbf{z}) \\right] $$

2. **KL Divergence**:
   - This term ensures that the posterior $q(\\mathbf{z} | \\mathbf{x})$ approximates the GP prior $p(\\mathbf{z})$ and penalizes deviations from the prior distribution. It regularizes the latent space, ensuring that the latent variables follow the Gaussian Process prior.

$$ \\mathcal{L}_{\\text{KL}} = D_{\\text{KL}} \\left( q(\\mathbf{z} | \\mathbf{x}) || p(\\mathbf{z}) \\right) $$

The total loss combines both of these terms:

$$ \\mathcal{L}_{\\text{total}} = \\mathcal{L}_{\\text{recon}} + \\mathcal{L}_{\\text{KL}} $$

### Training Process:

1. **Initialize the Model**:
   - The model is initialized with random parameters for both the encoder, the decoder, and the kernel function used in the GP prior.

2. **Sampling Latent Variables**:
   - During training, the model uses the **reparameterization trick** to sample latent variables $\\mathbf{z}$ from the approximate posterior $q(\\mathbf{z} | \\mathbf{x})$. The reparameterization trick enables the model to backpropagate through the stochastic sampling process. This is achieved by expressing $\\mathbf{z}$ as:

$$ \\mathbf{z} = \\mu + \\sigma \\cdot \\epsilon $$

   where $\\mu$ and $\\sigma$ are the mean and standard deviation predicted by the encoder, and $\\epsilon$ is a noise term sampled from a standard normal distribution $\\mathcal{N}(0, 1)$.

3. **Forward Pass**:
   - The encoder computes the mean $\\mu$ and variance $\\sigma$ for the latent variables $\\mathbf{z}$.
   - The latent variable $\\mathbf{z}$ is sampled using the reparameterization trick.
   - The decoder takes the latent variable $\\mathbf{z}$ and reconstructs the data $\\mathbf{x}$.

4. **Loss Calculation**:
   - After the forward pass, the model calculates both the reconstruction loss (how well the decoder reconstructs the input) and the KL divergence loss (how well the latent distribution approximates the GP prior).
   - The total loss is the sum of these two terms.

5. **Optimization**:
   - The model’s parameters (encoder, decoder, and the GP kernel) are updated by minimizing the total loss using an optimizer (typically **Adam** or **RMSprop**).
   - Backpropagation is used to compute the gradients of the total loss with respect to the model parameters, and the optimizer updates the weights to minimize the loss.

6. **Training Iterations**:
   - This process is repeated for several epochs, with the model gradually improving its ability to reconstruct the data and capture dependencies in the latent space.

7. **Final Model**:
   - After training, the model can generate new data by sampling from the latent space and passing it through the decoder. The GP prior ensures that the latent space is smooth and that generated samples have temporal or spatial coherence.

### Why Gaussian Process?
- **Capturing Complex Dependencies**: The GP prior allows the latent variables to capture complex, non-linear dependencies across time or space. This is crucial for data where observations are correlated, such as time-series data or spatial data (e.g., geospatial modeling).
- **Uncertainty Estimation**: The GP prior also provides uncertainty estimates. The latent variables are not just single points but are distributions, so the model can quantify uncertainty in its predictions. This is especially useful in forecasting and anomaly detection, where uncertainty plays a significant role in decision-making.

### Applications:
1. **Time-Series Forecasting**: By incorporating a GP prior in the latent space, GP-VAE captures temporal dependencies, making it ideal for predicting future values of time-series data (e.g., stock prices, weather).
2. **Anomaly Detection**: GP-VAE’s ability to model uncertainty allows it to detect anomalies effectively. If a data point is far from the expected distribution (reconstruction error and low likelihood), the model flags it as anomalous.
3. **Robust Image Generation**: GP-VAE can also be applied to generate images, with smooth transitions between features, benefiting from the GP prior’s ability to capture spatial correlations in the latent space.
4. **Geospatial Modeling**: For applications like environmental data analysis, where spatial correlations are critical (e.g., pollution prediction), GP-VAE models these dependencies effectively using Gaussian Processes.

### Summary:
GP-VAE merges the flexibility of **Variational Autoencoders (VAE)** with the power of **Gaussian Processes (GP)** to model complex, non-linear relationships in data, while also modeling uncertainty. Its ability to capture smooth dependencies in latent variables makes it well-suited for tasks involving time-series forecasting, anomaly detection, and generative modeling with uncertainty quantification. The training process uses **reparameterization**, **loss minimization**, and **backpropagation** to improve the model's performance over time.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.distributions as dist
from sklearn.datasets import make_moons
import matplotlib.pyplot as plt

In [ ]:
# 1. Define Encoder
class Encoder(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(Encoder, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3_mu = nn.Linear(64, latent_dim)
        self.fc3_logvar = nn.Linear(64, latent_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        mu = self.fc3_mu(x)
        logvar = self.fc3_logvar(x)
        return mu, logvar

In [ ]:
# 2. Define Decoder
class Decoder(nn.Module):
    def __init__(self, latent_dim, output_dim):
        super(Decoder, self).__init__()
        self.fc1 = nn.Linear(latent_dim, 64)
        self.fc2 = nn.Linear(64, 128)
        self.fc3 = nn.Linear(128, output_dim)

    def forward(self, z):
        z = torch.relu(self.fc1(z))
        z = torch.relu(self.fc2(z))
        return torch.sigmoid(self.fc3(z))

In [ ]:
# 3. Define GP-VAE Model
class GPVAE(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(GPVAE, self).__init__()
        self.encoder = Encoder(input_dim, latent_dim)
        self.decoder = Decoder(latent_dim, input_dim)
    
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std
    
    def forward(self, x):
        mu, logvar = self.encoder(x)
        z = self.reparameterize(mu, logvar)
        recon_x = self.decoder(z)
        return recon_x, mu, logvar, z

In [ ]:
# 4. Loss Function (ELBO)
def loss_function(recon_x, x, mu, logvar):
    # Reconstruction loss
    BCE = nn.BCELoss(reduction='sum')(recon_x, x)
    
    # KL divergence
    # KL(Q(z|x)||P(z)) = -0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    # where mu and logvar are the parameters of the approximate posterior q(z|x)
    # and sigma^2 is the variance (exp(logvar))
    # P(z) is the Gaussian prior, so it’s N(0, I)
    # KL term
    KL = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    
    return BCE + KL

In [ ]:
# 5. Create synthetic dataset (2D data for simplicity)
X, _ = make_moons(n_samples=1000, noise=0.1)
X = torch.tensor(X, dtype=torch.float32)

In [ ]:
# 6. Initialize model, optimizer
input_dim = X.shape[1]
latent_dim = 2  # Latent space dimensionality

model = GPVAE(input_dim, latent_dim)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# 7. Training Loop
num_epochs = 1000
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    recon_x, mu, logvar, z = model(X)
    loss = loss_function(recon_x, X, mu, logvar)

    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f"Epoch [{epoch}/{num_epochs}], Loss: {loss.item():.4f}")

In [ ]:
# 8. Visualizing Latent Space (2D)
model.eval()
with torch.no_grad():
    mu, logvar = model.encoder(X)
    z = model.reparameterize(mu, logvar)

# Plot the latent space
plt.scatter(z[:, 0].numpy(), z[:, 1].numpy(), alpha=0.5)
plt.title("Latent Space Representation")
plt.xlabel("Latent Variable 1")
plt.ylabel("Latent Variable 2")
plt.show()